In [1]:
import pandas as pd
import numpy as np
from astropy.io import fits

import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

In [2]:
#Maximum score of the sample with 80% of the lowest scores

filepath = '/Users/jimenagonzalez/research/DSPL/Citizen_science_project/Intersection/'
data_inter = pd.read_csv(filepath + 'intersection.csv')
data_inter = data_inter.dropna()  # Remove rows with NaN values
data_inter = data_inter.replace([np.inf, -np.inf], np.nan).dropna()

#80% of len(data_inter) is 602578

data_inter.sort_values(by='K_SCORE', inplace=True)
data_tmp = data_inter.head(602578)
print('Highest Rojas: ', data_tmp['K_SCORE'].iloc[-1])

data_inter.sort_values(by='score_sims', inplace=True)
data_tmp = data_inter.head(602578)
print('Highest Jacobs: ', data_tmp['score_sims'].iloc[-1])

data_inter.sort_values(by='SINGLE', inplace=True)
data_tmp = data_inter.head(602578)
print('Highest Gonzalez: ', data_tmp['SINGLE'].iloc[-1])

#maximum value of prob: 0.007

Highest Rojas:  0.006727251
Highest Jacobs:  0.0033
Highest Gonzalez:  0.001


In [3]:
data_zoo = pd.read_csv('data/complete_data.csv')

filepath = '/Users/jimenagonzalez/research/DSPL/Citizen_science_project/Intersection/data/in_sled2.fits'
hdu_list = fits.open(filepath)
images = hdu_list[1].data
data = pd.DataFrame(hdu_list[2].data)
hdu_list.close()

data.replace('None', np.nan, inplace=True)
data = data.astype({'COADD_OBJECT_ID': int, 'K_RA': float, 'K_DEC': float, 'K_SCORE': float, 'Y3_COADD_OBJECT_ID': int,
       'J_RA': float, 'J_DEC': float, 'score_sims': float, 'score_real': float, 'score_both': float, 'G_RA': float,
       'G_DEC': float, 'SINGLE': float, 'RING': float, 'SMOOTH': float, 'COMPANIONS': float, 'SDSS_SPIRALS': float,
       'DES_SPIRALS': float, 'CROWDED': float, 'ARTIFACTS': float, 'MOST_NEGATIVES': float, 'index': int, 'ra': float,
       'dec': float, 'name': str, 'score': float, 'image_sep': float, 'info': str, 'n_img': float, 'flag': str,
       'image_conf': str, 'lens_type': str, 'source_type': str, 'contaminant_type': str, 'papers': str})

#Getting the images
data.set_index('COADD_OBJECT_ID', inplace=True)
data_zoo.set_index('COADD_OBJECT_ID', inplace=True)
# Reindex 'data' to match the index of 'data_zoo'
data = data.reindex(data_zoo.index)
# Reset index if needed
data.reset_index(inplace=True)
data_zoo.reset_index(inplace=True)
images = images[data.index]

#Using calibrated score:
usernames = ['Krojas26', 'sgonzalezloz', 'curiosorus', 'ctortora', 'clarkguilty', 'alejandramelo', 'anupreeta']
scores_only = data_zoo[usernames]

def remove_min_max(row):
    valid_scores = row.dropna().tolist()
    if len(valid_scores) <= 2:
        # Not enough values to remove min and max
        return pd.Series([None] * max(0, len(valid_scores) - 2))
    trimmed = sorted(valid_scores)[1:-1]  # Remove lowest and highest
    return pd.Series(trimmed, index=[f'user{i+1}' for i in range(len(trimmed))])

data_zoo_tmp = data_zoo[usernames].apply(remove_min_max, axis=1)
data_zoo_tmp = pd.concat([data_zoo.drop(columns=usernames), data_zoo_tmp], axis=1)

new_usernames = ['user1', 'user2', 'user3', 'user4', 'user5']
data_zoo_tmp['calib'] = data_zoo_tmp[new_usernames].mean(axis=1)
#data_zoo_tmp['calib_std'] = data_zoo_tmp[new_usernames].std(axis=1)
#data_zoo_tmp['calib_range'] = data_zoo_tmp[new_usernames].max(axis=1) - data_zoo_tmp[new_usernames].min(axis=1)

data_zoo = data_zoo_tmp
data_zoo = data_zoo.drop(['average'], axis=1)
data_zoo = data_zoo.rename({'calib': 'average'}, axis=1)

In [4]:
# For most important paragraph. Most high-confidence candidates are recovered by the three works combined.
threshold_list = [0.8, 1.2, 1.8]

print(len(data_zoo))

# Candidates with ambiguous confidence - C
print(' ')
data_tmp = data_zoo[(data_zoo['average'] >= threshold_list[0]) & (data_zoo['average'] < threshold_list[1])]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate candidates with all ML scores < 0.8: {:.2f}%'.format(63/165*100))

# Candidates with moderate confidence - B
print(' ')
data_tmp = data_zoo[(data_zoo['average'] >= threshold_list[1]) & (data_zoo['average'] < threshold_list[2])]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate candidates with all ML scores < 0.8: {:.2f}%'.format(50/217*100))

# Candidates with high confidence - A
print(' ')
data_tmp = data_zoo[(data_zoo['average'] >= threshold_list[2])]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate candidates with all ML scores < 0.8: {:.2f}%'.format(3/61*100))

1651
 
165
63
Percentage moderate candidates with all ML scores < 0.8: 38.18%
 
217
50
Percentage moderate candidates with all ML scores < 0.8: 23.04%
 
61
3
Percentage moderate candidates with all ML scores < 0.8: 4.92%


In [5]:
# Studying the recovery individually: successive works recover more and more candidates

print(len(data_zoo))

# Candidates with moderate-high confidence
data_tmp = data_zoo[data_zoo['average'] >= threshold_list[1]]
print(len(data_tmp))

print(' ')
print(len(data_tmp[data_tmp['score_sims'] < 0.8]))
print('Percentage Jacobs ml score < 0.8: {:.2f}%'.format(192/278*100))

# Candidates with moderate confidence
print(' ')
print(len(data_tmp[data_tmp['K_SCORE'] < 0.8]))
print('Percentage Rojas ml score < 0.8: {:.2f}%'.format(134/278*100))

# Candidates with high confidence
print(' ')
print(len(data_tmp[data_tmp['SINGLE'] < 0.8]))
print('Percentage Gonzalez ml score < 0.8: {:.2f}%'.format(91/278*100))

1651
278
 
192
Percentage Jacobs ml score < 0.8: 69.06%
 
134
Percentage Rojas ml score < 0.8: 48.20%
 
91
Percentage Gonzalez ml score < 0.8: 32.73%


In [6]:
# Candidates with moderate-high confidence, for conclusion
print(' ')
data_tmp = data_zoo[data_zoo['average'] >= threshold_list[1]]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] < 0.8) & (data_tmp['K_SCORE'] < 0.8) & (data_tmp['SINGLE'] < 0.8)]))
print('Percentage moderate-high candidates with all ML scores < 0.8: {:.2f}%'.format(53/278*100))

print(' ')
data_tmp = data_zoo[data_zoo['average'] >= threshold_list[1]]
print(len(data_tmp))
print(len(data_tmp[(data_tmp['score_sims'] >= 0.8) | (data_tmp['K_SCORE'] >= 0.8) | (data_tmp['SINGLE'] >= 0.8)]))
print('Percentage moderate-high candidates with any ML score >= 0.8: {:.2f}%'.format(225/278*100))

 
278
53
Percentage moderate-high candidates with all ML scores < 0.8: 19.06%
 
278
225
Percentage moderate-high candidates with any ML score >= 0.8: 80.94%
